<a href="https://colab.research.google.com/github/zhengyanggao02/zhengyanggao_DTSC3020Fall2025/blob/main/Part_B_small_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import networkx as nx
import random
from collections import defaultdict

# 1) Small_map_edges
edges_small = [
    (0, 2, 11),
    (0, 4, 7),
    (2, 1, 4),
    (2, 3, 9),
    (3, 4, 3),
]

F = 0.8
TRIPS = 2000
TOP_K = 3
SEED = 42
random.seed(SEED)

In [10]:
# 2) Construct_the_graphics
G = nx.Graph()
for u, v, w in edges_small:
    G.add_edge(u, v, weight=w)

nodes = list(G.nodes())

In [11]:
# 3) Simulated_travel
pair_flow = defaultdict(int)
for _ in range(TRIPS):
    s, t = random.sample(nodes, 2)
    _ = nx.dijkstra_path(G, s, t, weight='weight')
    pair_flow[(s, t)] += 1

In [12]:
# 4) The_shortest_distance
def all_pairs_dist(G):
    dist = {}
    for src, d in nx.all_pairs_dijkstra_path_length(G, weight='weight'):
        for dst, val in d.items():
            dist[(src, dst)] = val
    return dist

dist = all_pairs_dist(G)

In [13]:
# 5) Candidate_new_path_benefits
benefit = {}
existing = set((min(u, v), max(u, v)) for u, v in G.edges())

for i in range(len(nodes)):
    for j in range(i + 1, len(nodes)):
        x, y = nodes[i], nodes[j]
        if (x, y) in existing:
            continue
        sp = dist[(x, y)]
        flow = pair_flow.get((x, y), 0) + pair_flow.get((y, x), 0)
        if flow <= 0:
            continue
        new_len = F * sp
        benefit[(x, y)] = (sp - new_len) * flow

In [14]:
# 6) Evaluate_new_paths
chosen = []
for _ in range(TOP_K):
    if not benefit:
        break
    (x, y), _ = max(benefit.items(), key=lambda kv: kv[1])
    sp = dist[(x, y)]
    new_w = F * sp
    G.add_edge(x, y, weight=new_w)
    chosen.append((x, y, new_w))
    dist = all_pairs_dist(G)
    benefit = {}

In [15]:
# 7) Before/After_average_distance
def avg_shortest(G):
    dist = all_pairs_dist(G)
    total, cnt = 0.0, 0
    for i in nodes:
        for j in nodes:
            if i == j: continue
            total += dist[(i, j)]
            cnt += 1
    return total / cnt

G_before = nx.Graph()
for u, v, w in edges_small:
    G_before.add_edge(u, v, weight=w)

G_after = G_before.copy()
for x, y, w in chosen:
    G_after.add_edge(x, y, weight=w)

avg_before = avg_shortest(G_before)
avg_after = avg_shortest(G_after)

In [16]:
# 8) Outcomes
print("="*60)
print("R3 —— Recommend a new path：")
if chosen:
    for x, y, w in chosen:
        print(f"  - ({x}, {y}), weight = {w:.2f}")
else:
    print("  - No candidates")

print("\nR4 —— Average shortest distance comparison：")
print(f"  - Before: {avg_before:.4f}")
print(f"  - After: {avg_after:.4f}")
print(f"  - Improve: {avg_before - avg_after:.4f}")

print("\nR5 —— Conclusion：")
if chosen:
    print("  - After the addition of new roads, the average shortest distance decreased and network connectivity improved.")
else:
    print("  - For new paths where no significant benefits have been found, one can attempt to adjust F or add TRIPS.")
print("="*60)

R3 —— Recommend a new path：
  - (4, 1), weight = 12.80

R4 —— Average shortest distance comparison：
  - Before: 10.0000
  - After: 9.6800
  - Improve: 0.3200

R5 —— Conclusion：
  - After the addition of new roads, the average shortest distance decreased and network connectivity improved.
